# Assignment - Weather and Citipy
Here, the flow is to find unique latitude and longitude coordinates. Then from Citipy, unique city names are identified.
For the list of Cities on weather api, key attitubute (temprature, humidity, wind speed and cloudiness) values are fetched.
I also have a scatter plot to show a range of coordinates across multiple locations.

Limitation: 
1. Some City names on Citipy dont exist on Weather app. To handle this, I am fetching 575 city names from which I will drop the cities not on weather app. Hence, I will have a little more than 500 city names in the list.
2. Inspite of restricting the duplicate city names, there are still a few. I have introduced a function later to delete duplicate cities.   

In [69]:
# Import all Dependencies
import random
import requests
import pandas as pd
import time as t
from time import strftime, gmtime
import openweathermapy.core as owm
import matplotlib.pyplot as plt
from config import api_key
from citipy import citipy
import numpy as np
import csv
from PIL import Image

In [70]:
import datetime

now = datetime.datetime.now()
idate = now.strftime("%Y-%m-%d")
itime = now.strftime("%H:%M")
filedate = now.strftime("%Y_%m_%d_%h_%m")

In [75]:
WorldCities_raw = pd.read_csv('worldcities.csv')
WorldCities_raw.dtypes

city           object
city_ascii     object
lat           float64
lng           float64
country        object
iso2           object
iso3           object
admin_name     object
capital        object
population    float64
id            float64
dtype: object

In [76]:
WorldCities = WorldCities_raw[['city_ascii',"lat",'lng','country']]

In [81]:
irange = WorldCities.shape
irange = irange[0]
irange


12897

In [82]:
WorldCities['Windspeed']=[0 for _ in range(0,irange)]
WorldCities['Humidity']=[0 for _ in range(0,irange)]
WorldCities['Cloudiness']=[0 for _ in range(0,irange)]
WorldCities['Temperature']=[0 for _ in range(0,irange)]
WorldCities['CurrentDate']=[idate for _ in range(0,irange)]
WorldCities['CurrentTime']=[itime for _ in range(0,irange)]
WorldCities= WorldCities.astype(str)
WorldCities.head()

C:\Users\sheel\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\sheel\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sheel\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,city_ascii,lat,lng,country,Windspeed,Humidity,Cloudiness,Temperature,CurrentDate,CurrentTime
0,Malisheve,42.4822,20.7458,Kosovo,0,0,0,0,2018-09-15,18:34
1,Prizren,42.2139,20.7397,Kosovo,0,0,0,0,2018-09-15,18:34
2,Zubin Potok,42.9144,20.6897,Kosovo,0,0,0,0,2018-09-15,18:34
3,Kamenice,42.5781,21.5803,Kosovo,0,0,0,0,2018-09-15,18:34
4,Viti,42.3214,21.3583,Kosovo,0,0,0,0,2018-09-15,18:34


In [83]:
WorldCities_subset = WorldCities.head(20)

In [85]:
Temperature = []
Humidity=[]
Cloudiness=[]
WindSpeed=[]

print(itime)

index = 1
for lat,lng in zip(WorldCities.lat,WorldCities.lng):
    url = "https://api.openweathermap.org/data/2.5/weather?lat="
    query_url = url + lat + "&lon=" + lng + "&units=imperial&appid=" + api_key
    response = requests.get(query_url).json()
    status_code = response['cod']
    
    if status_code == 200:
        Temperature.append(response['main']['temp'])
        Humidity.append(response['main']['humidity'])
        Cloudiness.append(response['clouds']['all'])
        WindSpeed.append(response['wind']['speed'])
        print(index, "City count , City Id: ", response["id"])
        print("City Name: " , response["name"])
        print(query_url)
        index += 1
        t.sleep(.30)
    else:
        Temperature.append('nan')
        Humidity.append('nan')
        Cloudiness.append('nan')
        WindSpeed.append('nan')

print(itime)

18:34
1 City count , City Id:  788239
City Name:  Malisevo
https://api.openweathermap.org/data/2.5/weather?lat=42.4822&lon=20.7458&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
2 City count , City Id:  786712
City Name:  Prizren
https://api.openweathermap.org/data/2.5/weather?lat=42.2139&lon=20.7397&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
3 City count , City Id:  783802
City Name:  Zubin Potok
https://api.openweathermap.org/data/2.5/weather?lat=42.9144&lon=20.6897&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
4 City count , City Id:  789227
City Name:  Kamenica
https://api.openweathermap.org/data/2.5/weather?lat=42.5781&lon=21.5803&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
5 City count , City Id:  784372
City Name:  Vitina
https://api.openweathermap.org/data/2.5/weather?lat=42.3214&lon=21.3583&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
6 City count , City Id:  785388
City Name:  Strpce
https://api.openweathermap.org/data/2.5/weather?

47 City count , City Id:  6940691
City Name:  Salh
https://api.openweathermap.org/data/2.5/weather?lat=13.6045&lon=44.0394&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
48 City count , City Id:  6940797
City Name:  As Safra
https://api.openweathermap.org/data/2.5/weather?lat=16.9398&lon=43.8498&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
49 City count , City Id:  76184
City Name:  Dhamar
https://api.openweathermap.org/data/2.5/weather?lat=14.5575&lon=44.3875&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
50 City count , City Id:  78428
City Name:  Amran
https://api.openweathermap.org/data/2.5/weather?lat=15.6594&lon=43.9439&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
51 City count , City Id:  80384
City Name:  Dhalie
https://api.openweathermap.org/data/2.5/weather?lat=13.6957&lon=44.7314&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
52 City count , City Id:  7792748
City Name:  At Tahalif
https://api.openweathermap.org/data/2.5/weather?lat=15.

93 City count , City Id:  979330
City Name:  Manor Gardens
https://api.openweathermap.org/data/2.5/weather?lat=-29.865&lon=30.98&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
94 City count , City Id:  989921
City Name:  Klerksdorp
https://api.openweathermap.org/data/2.5/weather?lat=-26.88&lon=26.62&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
95 City count , City Id:  1020605
City Name:  Bedelia
https://api.openweathermap.org/data/2.5/weather?lat=-27.97&lon=26.73&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
96 City count , City Id:  1011632
City Name:  De Aar
https://api.openweathermap.org/data/2.5/weather?lat=-30.65&lon=24.0&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
97 City count , City Id:  964090
City Name:  Prieska
https://api.openweathermap.org/data/2.5/weather?lat=-29.6595&lon=22.73&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
98 City count , City Id:  3361934
City Name:  Saldanha
https://api.openweathermap.org/data/2.5/weather?lat=-

139 City count , City Id:  897045
City Name:  Solwezi
https://api.openweathermap.org/data/2.5/weather?lat=-12.1796&lon=26.4&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
140 City count , City Id:  909137
City Name:  Lusaka
https://api.openweathermap.org/data/2.5/weather?lat=-15.4166&lon=28.2833&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
141 City count , City Id:  913323
City Name:  Kaoma
https://api.openweathermap.org/data/2.5/weather?lat=-14.7796&lon=24.8&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
142 City count , City Id:  918702
City Name:  Chipata
https://api.openweathermap.org/data/2.5/weather?lat=-13.6296&lon=32.64&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
143 City count , City Id:  898947
City Name:  Senanga
https://api.openweathermap.org/data/2.5/weather?lat=-16.1196&lon=23.27&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
144 City count , City Id:  898905
City Name:  Sesheke
https://api.openweathermap.org/data/2.5/weather?lat=-1

185 City count , City Id:  7418156
City Name:  Yeovil
https://api.openweathermap.org/data/2.5/weather?lat=-18.97&lon=32.65&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
186 City count , City Id:  890422
City Name:  Gweru
https://api.openweathermap.org/data/2.5/weather?lat=-19.45&lon=29.82&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
187 City count , City Id:  889191
City Name:  Karoi
https://api.openweathermap.org/data/2.5/weather?lat=-16.8196&lon=29.68&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
188 City count , City Id:  886763
City Name:  Masvingo
https://api.openweathermap.org/data/2.5/weather?lat=-20.0596&lon=30.82&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
189 City count , City Id:  894701
City Name:  Bulawayo
https://api.openweathermap.org/data/2.5/weather?lat=-20.17&lon=28.58&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
190 City count , City Id:  879431
City Name:  Victoria Falls
https://api.openweathermap.org/data/2.5/weather?lat=

...

12689 City count , City Id:  4574434
City Name:  Cheraw
https://api.openweathermap.org/data/2.5/weather?lat=34.6978&lon=-79.8988&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12690 City count , City Id:  4571470
City Name:  Bishopville
https://api.openweathermap.org/data/2.5/weather?lat=34.2202&lon=-80.2483&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12691 City count , City Id:  4597200
City Name:  Spartanburg
https://api.openweathermap.org/data/2.5/weather?lat=34.9437&lon=-81.9256&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12692 City count , City Id:  4580569
City Name:  Greenwood
https://api.openweathermap.org/data/2.5/weather?lat=34.1944&lon=-82.1541&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12693 City count , City Id:  4592912
City Name:  Ridgeland
https://api.openweathermap.org/data/2.5/weather?lat=32.4681&lon=-80.9178&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12694 City count , City Id:  4586940
City Name:  Melgrove (historical

12733 City count , City Id:  4292072
City Name:  Fort Wright
https://api.openweathermap.org/data/2.5/weather?lat=39.0334&lon=-84.5166&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12734 City count , City Id:  4301224
City Name:  Monticello
https://api.openweathermap.org/data/2.5/weather?lat=36.8404&lon=-84.8506&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12735 City count , City Id:  4287050
City Name:  Carrollton
https://api.openweathermap.org/data/2.5/weather?lat=38.68&lon=-85.1656&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12736 City count , City Id:  4294180
City Name:  Harrodsburg
https://api.openweathermap.org/data/2.5/weather?lat=37.7655&lon=-84.8474&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12737 City count , City Id:  4308163
City Name:  Shepherdsville
https://api.openweathermap.org/data/2.5/weather?lat=37.9841&lon=-85.7007&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12738 City count , City Id:  4291884
City Name:  Flatwoods
ht

12777 City count , City Id:  4283133
City Name:  Bardstown
https://api.openweathermap.org/data/2.5/weather?lat=37.8175&lon=-85.455&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12778 City count , City Id:  4301732
City Name:  Mount Washington
https://api.openweathermap.org/data/2.5/weather?lat=38.0427&lon=-85.5553&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12779 City count , City Id:  4304232
City Name:  Pikeville
https://api.openweathermap.org/data/2.5/weather?lat=37.4808&lon=-82.5261&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12780 City count , City Id:  4290988
City Name:  Elizabethtown
https://api.openweathermap.org/data/2.5/weather?lat=37.7033&lon=-85.8771&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12781 City count , City Id:  4303951
City Name:  Perry County
https://api.openweathermap.org/data/2.5/weather?lat=37.2583&lon=-83.1977&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12782 City count , City Id:  4289323
City Name:  Cynthian

12821 City count , City Id:  5735724
City Name:  Lake Oswego
https://api.openweathermap.org/data/2.5/weather?lat=45.4129&lon=-122.7002&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12822 City count , City Id:  5744166
City Name:  Ontario
https://api.openweathermap.org/data/2.5/weather?lat=44.0259&lon=-116.976&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12823 City count , City Id:  5751600
City Name:  Sheridan
https://api.openweathermap.org/data/2.5/weather?lat=45.0956&lon=-123.3972&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12824 City count , City Id:  5718601
City Name:  Central Point
https://api.openweathermap.org/data/2.5/weather?lat=42.3764&lon=-122.911&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12825 City count , City Id:  5713587
City Name:  Bend
https://api.openweathermap.org/data/2.5/weather?lat=44.0562&lon=-121.3087&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12826 City count , City Id:  5740898
City Name:  Milton-Freewater
htt

12865 City count , City Id:  5731777
City Name:  Hood River
https://api.openweathermap.org/data/2.5/weather?lat=45.7092&lon=-121.52600000000001&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12866 City count , City Id:  5720727
City Name:  Corvallis
https://api.openweathermap.org/data/2.5/weather?lat=44.5697&lon=-123.2781&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12867 City count , City Id:  5755890
City Name:  Talent
https://api.openweathermap.org/data/2.5/weather?lat=42.2405&lon=-122.7808&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12868 City count , City Id:  5754464
City Name:  Stayton
https://api.openweathermap.org/data/2.5/weather?lat=44.803000000000004&lon=-122.79799999999999&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12869 City count , City Id:  5727032
City Name:  Florence
https://api.openweathermap.org/data/2.5/weather?lat=43.9916&lon=-124.1065&units=imperial&appid=9c4bb9576cd41932b37e7ede0d5c7753
12870 City count , City Id:  5744345
Ci

In [86]:
Cloudiness

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 20,
 12,
 0,
 12,
 56,
 8,
 0,
 0,
 56,
 0,
 0,
 36,
 56,
 12,
 0,
 0,
 0,
 92,
 0,
 88,
 68,
 100,
 8,
 0,
 0,
 44,
 0,
 75,
 0,
 0,
 0,
 12,
 0,
 8,
 24,
 0,
 0,
 12,
 88,
 0,
 20,
 0,
 76,
 12,
 0,
 0,
 12,
 0,
 36,
 0,
 0,
 0,
 36,
 0,
 0,
 100,
 0,
 12,
 8,
 12,
 32,
 12,
 80,
 0,
 92,
 8,
 92,
 80,
 0,
 0,
 20,
 0,
 44,
 8,
 32,
 8,
 0,
 0,
 0,
 88,
 0,
 75,
 80,
 32,
 0,
 0,
 75,
 32,
 0,
 36,
 8,
 44,
 44,
 80,
 0,
 48,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 12,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 44,
 92,
 44,
 92,
 12,
 12,
 76,
 32,
 0,
 0,
 40,
 48,
 80,
 48,
 88,
 0,
 92,
 0,
 0,
 44,
 44,
 80,
 8,
 64,
 0,
 44,
 76,
 0,
 12,
 0,
 0,
 0,
 6

In [87]:
WorldCities["Temperature"] = Temperature
WorldCities["Humidity"] = Humidity
WorldCities["Cloudiness"] = Cloudiness
WorldCities["WindSpeed"] = WindSpeed

In [88]:
WorldCities.to_csv(f'Results_{filedate}.csv', sep=',', encoding='utf-8',index=False)